In [1]:
"""
UNet configs comparison: from batchflow, from batchflow with fix, mine
"""

'\nUNet configs comparison: from batchflow, from batchflow with fix, mine\n'

In [2]:
import sys
sys.path.append("..")

In [3]:
import os
import shutil

# import warnings
# warnings.filterwarnings('ignore')

# from matplotlib import pyplot as plt
# %matplotlib inline

# from tensorflow import logging
# logging.set_verbosity(logging.ERROR)
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import numpy as np
import batchflow
from batchflow import Pipeline, B, C, V, D
from batchflow.opensets import PascalSegmentation
from batchflow.research import Research, Option, Domain, Results, PrintLogger, RP, REP, KV
from batchflow.models.metrics import Loss
from batchflow.models.torch import UNet, EncoderDecoder

In [4]:
BATCH_SIZE = 64
ITERATIONS = 1#1000
N_REPS = 1#10
IMAGE_SHAPE = (160, 160)

dataset = PascalSegmentation(bar='n')

In [5]:
def clear_previous_results(res_name):
    if os.path.exists(res_name):
        shutil.rmtree(res_name)

# Models configs

In [6]:
num_classes = 22

task_config = {
    'inputs/targets/classes': num_classes,  
    'head/layout': 'c',
    'head/filters': num_classes,
    'head/kernel_size': 1,
    'loss': 'ce',
    'optimizer': 'Adam'
}

# UNet from batchflow
UNet_bf = {'inputs': {'targets': {'classes': 22}},
 'placeholder_batch_size': 2,
 'device': None,
 'benchmark': True,
 'profile': False,
 'microbatch': None,
 'sync_frequency': 1,
 'optimizer': 'Adam',
 'decay': None,
 'amp': True,
 'sam_rho': 0.0,
 'sam_individual_norm': True,
 'order': ['initial_block', 'body', 'head'],
 'initial_block': {},
 'body': {'encoder': {'downsample': {'layout': 'p',
    'pool_size': 2,
    'pool_strides': 2},
   'num_stages': 4,
   'order': ['block', 'skip', 'downsampling'],
   'blocks': {'base': batchflow.models.torch.blocks.DefaultBlock,
    'layout': 'cna cna',
    'kernel_size': 3,
    'filters': [64, 128, 256, 512]}},
  'decoder': {'skip': True,
   'num_stages': None,
   'factor': None,
   'upsample': {'layout': 'tna'},
   'combine': {'op': 'concat', 'leading_index': 1},
   'order': ['upsampling', 'combine', 'block'],
   'blocks': {'base': batchflow.models.torch.blocks.DefaultBlock,
    'layout': 'cna cna',
    'kernel_size': 3,
    'filters': [512, 256, 128, 64]}},
  'embedding': {'base': batchflow.models.torch.blocks.DefaultBlock,
   'layout': 'cna cna',
   'kernel_size': 3,
   'filters': 1024}},
 'head': {'layout': 'c',
  'filters': 22,
  'kernel_size': 1,
  'target_shape': None,
  'classes': 22,
  'units': 22},
 'common': {'data_format': 'channels_first'},
 'predictions': None,
 'output': None,
 'loss': 'ce'}

# fixed UNet from batchflow
config_bf_with_fix = UNet_bf.copy()
config_bf_with_fix['body/decoder/upsample'] = dict(layout='tna', filters=[512, 256, 128, 64])

In [7]:
# my_UNet
downsample_depth = 4

my_config = {
    'body/encoder': {
        'num_stages': downsample_depth,
        'order': ['block', 'skip', 'downsampling']
    },    
    'body/encoder/blocks': {
        'layout': 'cna cna',
        'filters': [32*pow(2, i) for i in range(1, downsample_depth+1)]
    },
    'body/encoder/downsample': {
        'layout': 'p'
    },    
    
    'body/embedding': {
        'layout': 'cna cna', 
        'filters': [64*pow(2, downsample_depth), 64*pow(2, downsample_depth)]
    },   

    'body/decoder': {
        'num_stages': downsample_depth,
        'order': ['upsampling', 'combine', 'block']
    },
    'body/decoder/upsample': {
        'layout': 'tna',
        'filters': [32*pow(2, i) for i in range(downsample_depth, -1, -1)]
    },
    'body/decoder/combine': {
        'op': 'concat'
    },
    'body/decoder/blocks': {
        'layout': 'cna cna',
        'filters': [64*pow(2, i-1) for i in range(downsample_depth, -1, -1)]
    }
}
my_config.update(task_config)

# train

In [8]:
def process_mask(x):
    x = np.squeeze(x)
    np.place(x, x==255, 21)
    return x

train_ppl = (dataset.train.p
    .init_variable('train_loss', [])
    .init_model('dynamic', C('model'), 'model', config=C('config'))
    .resize(size=IMAGE_SHAPE, src=['images', 'labels'], dst=['images', 'labels'])
    .to_array(channels='first', src=['images', 'labels'], dst=['images', 'labels'])
    .process_mask(B('labels'), save_to=B('labels'))
    .train_model('model', B('images'), B('labels'),
                fetches='loss', save_to=V('train_loss', mode='a'))
    .run_later(BATCH_SIZE, shuffle=True, n_epochs=None)
)



In [9]:
# configs = [KV(task_config, "config_bf"), KV(config_bf_with_fix, "config_bf_with_fix"), KV(my_config, "my_config")]
# domain = Option('model', [UNet, UNet, EncoderDecoder]) @ Option('config', configs)
# list(domain.iterator)

configs = [KV(my_config, "my_config")]
domain = Option('model', [EncoderDecoder]) @ Option('config', configs)
list(domain.iterator)

[ConfigAlias({'model': 'EncoderDecoder', 'config': 'my_config', 'repetition': '0'})]

In [10]:
# research = (Research()
#             .init_domain(domain, n_reps=N_REPS)
#             .add_pipeline(train_ppl, variables='loss_history', name='train_ppl', logging=True))

# res_name='UNet_pascal_research'
# clear_previous_results(res_name)

In [11]:
# research.run(n_iters=ITERATIONS, name=res_name, bar=True, workers=1, devices=[2])

In [12]:
# results = research.load_results().df
# results.info()

In [13]:
# results.head()

# performance

In [14]:
# add test pipeline for getting metrics with param execute last

In [15]:
test_ppl = (dataset.test.p
                .import_model('model', C('import_from'))
                .init_variable('metrics')
                .init_variable('predictions')
                .resize(size=IMAGE_SHAPE, src=['images', 'labels'], dst=['images', 'labels'])
                .to_array(channels='first', src=['images', 'labels'], dst=['images', 'labels'])
                .process_mask(B('labels'), save_to=B('labels'))                
                .predict_model('model', B('images'), fetches='predictions',
                               save_to=V('predictions'))
                .run_later(BATCH_SIZE, shuffle=False, n_epochs=1)
            )

In [ ]:
TEST_EXECUTE_FREQ = 'last'

res_name = 'UNet_pascal_train_test_research'
clear_previous_results(res_name)

# research = (Research()
#             .init_domain(domain, n_reps=N_REPS)
#             .add_pipeline(train_ppl, variables='train_loss', name='train_ppl')
#             .add_pipeline(test_ppl, variables='predictions', name='test_ppl',
#                         execute=TEST_EXECUTE_FREQ, run=True, import_from=RP('train_ppl')))


research = (Research()
            .init_domain(domain, n_reps=N_REPS)
            .add_pipeline(train_ppl, variables='train_loss', name='train_ppl')
            .add_pipeline(test_ppl, name='test_ppl',
                         execute=TEST_EXECUTE_FREQ, run=True, import_from=RP('train_ppl'))
            .get_metrics(pipeline='test_ppl', metrics_var='metrics', metrics_name='accuracy',
                         returns='accuracy', execute=TEST_EXECUTE_FREQ))



research.run(n_iters=ITERATIONS, name=res_name, bar=True)

Research UNet_pascal_train_test_research is starting...


Domain updated: 0: : 0it [00:00, ?it/s]

In [ ]:
research.load_results().df.head()

In [ ]:

from batchflow.utils import plot_results_by_config

plot_results_by_config(df, {'train_ppl': 'train_loss', 
                            'test_ppl_test_loss': 'test_loss', 
                            'test_ppl_metric_class': 'accuracy'},
                       figsize=(20, 15))